In [ ]:
!pip install ultralytics

In [ ]:
import os
import shutil
import random

!pip install tqdm --upgrade
from tqdm.notebook import tqdm

In [ ]:
from ultralytics import YOLO

In [ ]:
model = YOLO("yolov8m.pt")

In [ ]:
#!yolo task=detect mode=predict model=/content/runs/detect/train2/weights/best.pt  data= meta.yaml imgsz=640

In [ ]:

from ultralytics import YOLO

class TrafficLightYOLOv8(nn.Module):
    def __init__(self):
        super(TrafficLightYOLOv8, self).__init__()
        self.model = YOLO('model')

    def forward(self, x):
        return self.model(x)

class MAML:
    def __init__(self, model, lr_inner=0.01, lr_outer=0.001):
        self.model = model
        self.lr_inner = lr_inner
        self.lr_outer = lr_outer
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.lr_outer)

    def inner_loop(self, support_data, support_labels):
        """Inner loop fine-tuning"""
        inner_optimizer = optim.Adam(self.model.parameters(), lr=self.lr_inner)
        criterion = nn.CrossEntropyLoss()

        for _ in range(1):  # Single step update
            outputs = self.model(support_data)
            loss = criterion(outputs, support_labels)
            inner_optimizer.zero_grad()
            loss.backward()
            inner_optimizer.step()

        return loss

    def outer_loop(self, tasks):
        """Outer loop meta-optimization"""
        self.model.train()

        meta_loss = 0
        for support_data, support_labels, query_data, query_labels in tasks:
            # Clone model for task-specific fine-tuning
            task_model = TrafficLightYOLOv8()
            task_model.load_state_dict(self.model.state_dict())

            # Inner loop
            self.inner_loop(support_data, support_labels)

            # Query set evaluation
            outputs = task_model(query_data)
            criterion = nn.CrossEntropyLoss()
            loss = criterion(outputs, query_labels)
            meta_loss += loss

        # Meta-optimization step
        self.optimizer.zero_grad()
        meta_loss.backward()
        self.optimizer.step()

        return meta_loss.item()

# Data preparation
# Placeholder code for data loading
def load_data():
    # Assume you have a data loading function that returns support and query sets
    tasks = []
    for _ in range(10):  # Example with 10 tasks
        support_data = torch.randn(5, 3, 224, 224)  # 5 support examples
        support_labels = torch.randint(0, 3, (5,))  # 3 classes
        query_data = torch.randn(5, 3, 224, 224)  # 5 query examples
        query_labels = torch.randint(0, 3, (5,))  # 3 classes
        tasks.append((support_data, support_labels, query_data, query_labels))
    return tasks

# Training
if __name__ == "__main__":
    model = TrafficLightYOLOv8()
    maml = MAML(model)

    tasks = load_data()  # Load your tasks here
    for epoch in range(100):  # Number of epochs
        meta_loss = maml.outer_loop(tasks)
        print(f"Epoch {epoch+1}, Meta Loss: {meta_loss}")


In [ ]:
!yolo task=detect mode=predict model=/content/foggy_model_meta.pt conf=0.4 source=/content/IMG-20230220-WA0036_jpg.rf.a3b8b3d823a8f3210bd9d4f1dc403407.png show=True